In [56]:
# manipulation des données
import numpy as np
import pandas as pd

# matplotlib et seaborn pour les représentations graphiques
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
df = pd.read_csv("../Data/2.sample_dataset.csv")
df = df.dropna(subset=['words'])

In [58]:
batch_1 = df.sample(1000)

In [59]:
batch_1['target'].value_counts()

target
0    515
1    485
Name: count, dtype: int64

In [60]:
import torch
from transformers import AutoModel, AutoTokenizer

In [82]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [83]:
text = 'I am so happy to see you at this evening!'

In [76]:
text = 'I am so happy to see you at this evening!'
tokens = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")

In [77]:
outputs = bertweet(tokens)
logits = outputs.last_hidden_state

In [84]:
from scipy.special import softmax

In [85]:
labels = ['Negative', 'Neutral', 'Positive']

# sentiment analysis
encoded_tweet = tokenizer(text, return_tensors='pt')
output = bertweet(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

Negative [7.72545682e-05 7.71932100e-05 8.43600355e-05 5.98084735e-05
 7.24764541e-05 6.10682691e-05 7.04932536e-05 6.75996853e-05
 7.32230546e-05 6.11402284e-05 7.56897571e-05 8.89040966e-05
 7.68745231e-05 6.78662473e-05 6.50031725e-05 5.97713261e-05
 7.65511286e-05 7.94790249e-05 1.00997102e-04 1.00720026e-04
 7.99291520e-05 3.35034274e-05 6.60709120e-05 6.14701203e-05
 7.96413879e-05 6.13674056e-05 7.05234415e-05 6.00313979e-05
 5.74419864e-05 7.58035821e-05 7.25310456e-05 9.17117941e-05
 4.47180755e-05 5.44028408e-05 1.04835388e-04 6.46141343e-05
 7.19554519e-05 7.41391996e-05 9.45953871e-05 7.51319021e-05
 8.08186742e-05 5.32518352e-05 8.89372677e-05 5.25089563e-05
 8.36387044e-05 6.98679360e-05 8.32034639e-05 6.27435074e-05
 9.52660994e-05 8.27984986e-05 7.59521427e-05 9.40113969e-05
 7.45744546e-05 7.48314342e-05 5.70206576e-05 3.54170043e-05
 6.18549748e-05 1.07577718e-04 1.00742953e-04 9.95799855e-05
 6.20244828e-05 6.73120448e-05 9.44128260e-05 9.40033206e-05
 1.18210017e-04

IndexError: list index out of range

In [78]:
from torch.nn.functional import softmax

In [79]:
probs = softmax(logits, dim=1)
print(probs)

tensor([[[0.0506, 0.0742, 0.0666,  ..., 0.0662, 0.0692, 0.0705],
         [0.0818, 0.0678, 0.0852,  ..., 0.0678, 0.0899, 0.1230],
         [0.0415, 0.0570, 0.0775,  ..., 0.0726, 0.0935, 0.0740],
         ...,
         [0.0903, 0.0535, 0.0661,  ..., 0.0586, 0.0755, 0.0698],
         [0.0831, 0.0851, 0.0766,  ..., 0.0790, 0.0978, 0.0665],
         [0.0502, 0.0751, 0.0683,  ..., 0.0639, 0.0715, 0.0717]]],
       grad_fn=<SoftmaxBackward0>)


In [80]:
average_probs = torch.mean(probs, dim=0)
print(average_probs)

tensor([[0.0506, 0.0742, 0.0666,  ..., 0.0662, 0.0692, 0.0705],
        [0.0818, 0.0678, 0.0852,  ..., 0.0678, 0.0899, 0.1230],
        [0.0415, 0.0570, 0.0775,  ..., 0.0726, 0.0935, 0.0740],
        ...,
        [0.0903, 0.0535, 0.0661,  ..., 0.0586, 0.0755, 0.0698],
        [0.0831, 0.0851, 0.0766,  ..., 0.0790, 0.0978, 0.0665],
        [0.0502, 0.0751, 0.0683,  ..., 0.0639, 0.0715, 0.0717]],
       grad_fn=<MeanBackward1>)


In [81]:
predicted_class = torch.argmax(average_probs).item()

# Afficher la classe prédite
print(f"La classe prédite est : {predicted_class}")

La classe prédite est : 445


In [66]:
text = 'I am so happy to see you at this evening!'
inputs = tokenizer(text, return_tensors="pt")
outputs = bertweet(**inputs)

In [67]:
tokens = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")

In [63]:
from torch.nn.functional import softmax

In [64]:
probs = softmax(outputs.logits, dim=1)
print(probs)

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [44]:
tokenized = batch_1['words'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [45]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [46]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 37)

In [47]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = bertweet(input_ids, attention_mask=attention_mask)

In [48]:
features = last_hidden_states[0][:,0,:].numpy()

In [50]:
labels = batch_1['target']

In [53]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [54]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

/Users/gaeldelescluse/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [55]:
lr_clf.score(test_features, test_labels)

0.5

In [20]:
import torch
from transformers import AutoModel, AutoTokenizer

In [21]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [36]:
from transformers import AutoModel, AutoTokenizer

# Charger le modèle BERTweet et le tokenizer
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# Texte d'exemple
text = "It was an excellent movie, I loved every moment!"

# Tokenisation et encodage avec le tokenizer BERTweet
tokens = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")

# Utilisation du modèle pour obtenir les logits
outputs = bertweet(tokens)
logits = outputs.last_hidden_state  # Si vous souhaitez obtenir les embeddings finaux

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [86]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# tweet = "@MehranShakarami today's cold @ home 😒 https://mehranshakarami.com"
tweet = 'Great content! subscribed 😉'

# precprcess tweet
tweet_words = []

for word in tweet.split(' '):
    if word.startswith('@') and len(word) > 1:
        word = '@user'
    
    elif word.startswith('http'):
        word = "http"
    tweet_words.append(word)

tweet_proc = " ".join(tweet_words)

# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

# sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
output = model(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

Negative 0.0020959747
Neutral 0.01897672
Positive 0.97892725


In [87]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# tweet = "@MehranShakarami today's cold @ home 😒 https://mehranshakarami.com"
tweet = 'Great content! subscribed 😉'

# precprcess tweet
tweet_words = []

#for word in tweet.split(' '):
#    if word.startswith('@') and len(word) > 1:
#        word = '@user'
#   
#    elif word.startswith('http'):
#        word = "http"
#    tweet_words.append(word)


#tweet_proc = " ".join(tweet_words)
tweet_proc = tweet

# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

# sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
output = model(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

Negative 0.0020959747
Neutral 0.01897672
Positive 0.97892725


In [92]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

tweet = 'Great content! subscribed 😉'
tweet_proc = tweet
#roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
#bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
#tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

labels = ['Negative', 'Neutral', 'Positive']

# sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
output = model(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

Negative 0.0020959747
Neutral 0.01897672
Positive 0.97892725


In [93]:
print(scores)

[0.00209597 0.01897672 0.97892725]


In [94]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

tweet = 'Great content! subscribed 😉'
tweet_proc = tweet
#roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
#bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
#tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

labels = ['Negative', 'Neutral', 'Positive']

# sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
output = model(**encoded_tweet)

In [97]:
from scipy.special import softmax

logits = outputs.last_hidden_state
probs = softmax(logits, dim=1)
print(probs)

TypeError: softmax() got an unexpected keyword argument 'dim'

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Charger le modèle de tokenisation
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Charger le modèle de classification de sentiment
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Tokeniser la phrase
text = "Hello, my dog is cute"
inputs = tokenizer(text, return_tensors="pt")

# Effectuer la prédiction
outputs = model(**inputs)

# Obtenir les probabilités de chaque classe
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Déterminer la classe prédite (positif ou négatif)
predicted_class = torch.argmax(probs, dim=-1).item()

# Afficher le résultat
if predicted_class == 1:
    print("Sentiment positif")
else:
    print("Sentiment négatif")


/Applications/anaconda3/envs/Projet7bisEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sentiment négatif
